In [14]:
import datetime

import os

os.getcwd()
if '.git' not in os.listdir():
    os.chdir('./../../../../')
os.listdir()

import fasttext
import keras_core as keras
import numpy as np
import tensorflow as tf
from keras_core.models import load_model
from sklearn.model_selection import train_test_split
from tensorboard.plugins.hparams import api as hp
import tensorflow_hub as hub
import tensorflow_text as text


## Configurations

In [15]:
params_grid = {
    "model": "mlp",
    "embedder": "bert",
    "embedding_data": "small_bert_bert_en_uncased_L-4_H-512_A-8",
    "dataset": "fr_hf.csv",
    "batch_size": 500
}

In [16]:
data_folder_src = 'ml/ml_core/data/processed/'
ml_core_folder_path = 'ml/ml_core/'
data_path_src = f'{data_folder_src}{params_grid["dataset"]}'
date_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
experiment_name = f'deepwoke_{params_grid["embedder"]}_{params_grid["model"]}_{params_grid["dataset"]}_{date_str}'

log_dir = f"{ml_core_folder_path}log/fit/{experiment_name}"
model_weight_dst = f'{ml_core_folder_path}model_weights/{experiment_name}-model.keras'

In [17]:

vector_path = f'{ml_core_folder_path}embedded_vector/{params_grid["embedding_data"]}_{params_grid["dataset"]}'
vector_x_dst = f'{vector_path}.x.npy'
vector_y_dst = f'{vector_path}.y.npy'

# loading dataset

In [18]:
X = np.load(vector_x_dst)
y = np.load(vector_y_dst)
X

array([[ 0.94331115,  0.91675884,  0.27863583, ..., -0.6207063 ,
         0.38985655, -0.40558928],
       [ 0.9879986 ,  0.9389037 , -0.02450722, ..., -0.6357726 ,
        -0.0533436 , -0.8712714 ],
       [ 0.9917565 ,  0.9596368 ,  0.23419988, ..., -0.5770884 ,
         0.01479267, -0.84448135],
       ...,
       [ 0.85382503,  0.97301006,  0.05384289, ..., -0.06826842,
        -0.16658394, -0.20173083],
       [ 0.99016047,  0.77991027,  0.01531475, ...,  0.46481106,
         0.03767   , -0.1936145 ],
       [ 0.992991  ,  0.90109193, -0.03615341, ...,  0.34522945,
        -0.12749028, -0.8523576 ]], dtype=float32)

# Model

## callbacks

In [19]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='loss'
)


In [20]:

class EarlyStoppingLogging(keras.callbacks.Callback):
    def __init__(self, early_stopping_callback, log_dir):
        super().__init__()
        self.early_stopping = early_stopping_callback
        self.stopped_epoch = 0
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        if self.early_stopping.stopped_epoch > 0:
            self.stopped_epoch = self.early_stopping.stopped_epoch
            with self.writer.as_default():
                tf.summary.scalar('early_stopping_epoch', self.stopped_epoch, step=epoch)
                self.writer.flush()


In [21]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_logging_callback = EarlyStoppingLogging(early_stopping, log_dir)

In [22]:
callback = [
    early_stopping,
    tensorboard_callback,
    hp.KerasCallback(log_dir, params_grid),
    early_stopping_logging_callback,
]

### training the model

In [23]:
from ml.ml_core.src.model.model_factory import model_factory

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

model = model_factory(params_grid['model'], X_train.shape[1], 1)
print(model.summary)

tf.compat.v1.global_variables_initializer()

512 1
<bound method Model.summary of <Functional name=functional_3, built=True>>


In [24]:
model.fit(X_train,
          y_train,
          epochs=500,
          batch_size=params_grid['batch_size'],
          callbacks=callback,
          validation_data=(X_test, y_test)
          )

Epoch 1/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 97ms/step - accuracy: 0.7522 - loss: 0.5570 - val_accuracy: 0.7588 - val_loss: 0.5224
Epoch 2/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7631 - loss: 0.5104 - val_accuracy: 0.7637 - val_loss: 0.5102
Epoch 3/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7648 - loss: 0.5030 - val_accuracy: 0.7656 - val_loss: 0.5010
Epoch 4/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7706 - loss: 0.4898 - val_accuracy: 0.7757 - val_loss: 0.4900
Epoch 5/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7717 - loss: 0.4884 - val_accuracy: 0.7737 - val_loss: 0.4879
Epoch 6/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7763 - loss: 0.4784 - val_accuracy: 0.7812 - val_loss: 0.4788
Epoch 7/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7786 - loss: 0.4751 - val_accuracy: 0.7759 - val_loss: 0.4838
Epoch 8/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7795 - loss: 0.4695 - val_accuracy: 0.7847 - 

# save model

In [12]:
model.save(model_weight_dst)

### testing with custom sentence

In [17]:
#model_weight_dst = f'../../model_weights/fasttext_nlp_20240613-131608-model.keras'
new_model = load_model(model_weight_dst)

In [18]:
ft = fasttext.load_model('ml/ml_core/data/embedding_data/cc.fr.300.bin')


#from ml.ml_core.src.embedding.embedding_utils import text_to_vector

text_nice = [
    "la bonjour je suis un type sympas",
    "bonjour ça va les gens",
    "j'aime pas trop ce mec",
    "cette meuf est cool",
    "les meuf sont trop cool",
    "les mecs sont des gens stylé",
    "le mec est stylé",
    "nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse",
    "les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste",
    "Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions"

]

text_no_nice = [
    "regarde moi cette connasse",
    "les femme leur place c'est à la cuisine",
    "femme au volant, accident",
    "les femmes on peux pas leur faire confiance",
    "Pour une transphobe son passing est pas ouf",
    "Elle l’ouvre quand même beaucoup celle là pour quelqu’un qu’est pas capable d’éduquer ses enfants",
    "Dis donc, c’est pas parce que t’es mal baisée que tu peux tout te permettre ????",
    "Moi ce que j’aime bien chez les filles comme toi c’est que t’es pas du genre prise de tête",
    "les femmes c'est comme le fromage, quand elles puent c'est qu'elles sont bonnes",
    "Oh les grosses tepus",
    "Faut pas t'étonner si tu te fais violer vu comment t'es habillée",
    "Tu deverais avoir des enfants maintenant avant que ce soit trop tard",
    "Vous seriez une collaboratrice idéal pour notre entreprise, malheureusement nous cherchons quelqu’un disponible sur le long terme, et nous craignons qu’à votre âge des changements familiaux ne vous permettent plus de vous investir autant que besoins"
]
wordsvec_nice = [np.expand_dims(text_to_vector(text, ft), axis=0) for text in text_nice]
wordsvec_no_nice = [np.expand_dims(text_to_vector(text, ft), axis=0) for text in text_no_nice]

print("nice output")
for i in range(0, len(wordsvec_nice)):
    prediction = new_model.predict(wordsvec_nice[i], verbose=False)
    print(f"{text_nice[i]} : {prediction > 0.5}")
    print(f"{prediction}")

print("\n\nnot nice output")
for i in range(0, len(wordsvec_no_nice)):
    prediction = new_model.predict(wordsvec_no_nice[i], verbose=False)
    print(f"{text_no_nice[i]} : {prediction > 0.5}")
    print(f"{prediction}")

nice output
la bonjour je suis un type sympas : [[False]]
[[0.00716593]]
bonjour ça va les gens : [[False]]
[[0.00271189]]
j'aime pas trop ce mec : [[False]]
[[1.9822155e-05]]
cette meuf est cool : [[False]]
[[7.958688e-08]]
les meuf sont trop cool : [[False]]
[[3.9379524e-11]]
les mecs sont des gens stylé : [[False]]
[[1.0103315e-05]]
le mec est stylé : [[False]]
[[3.5561166e-06]]
nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse : [[False]]
[[0.01031799]]
les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste : [[ True]]
[[0.9999633]]
Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions : [[ True]]
[[0.99984384]]


not nice output
regarde moi cette connasse : [[False]]
[[0.02953639]]
les femme leur place c'est à la cuisine : [[False]]
[[0.01907552]]
femme au volant, accident : [[False]]
[[0.00229132]]
les femmes on peux pas leur faire confiance : 